# Exporting an AstroID REDCap build and importing into MSSQL

Created by: Benjamin Green - 03.10.2024;
Last Edit 06.14.2024;
Python 3.10.14

To get started, pip install the corresponding packages and import them.

In [1]:
import io
import requests
import numpy
import pandas
import geopandas
from sqlalchemy import create_engine, MetaData
import sqlalchemy


Next, set up the input variables below.

In [ ]:
#
token = '' # A REDCap API token
username = '' # A username for an SQL database
password = '' # A password for an SQL database
ip = '' # An ip address for an SQL database
port = '' # A port for an SQL database
key_1 = 'patient_tier' # The first tier in the cascading structure as described in the AstroID format
pt_id = 'astropt' # The patient identifier for the cascading structure as described in the AstroID format

Now, run the following cell. This will create the appropriate functions and classes to read the data from REDCap, organize them, and load them into a SQL database. It will not run any of the code, just set things up.

In [3]:
def create_request_input(token, content, beginTime = '2020-01-16 15:10', endTime = ''):
    """
    create input JSON for requests.post

    Args:
        token: redcap token
        content: redcap table/form to use
    """
    if content == 'log':
        return {
            'token': token,
            'content': content,
            'logtype': '',
            'user': '',
            'record': '',
            'beginTime': beginTime,
            'endTime': endTime,
            'format': 'csv',
            'returnFormat': 'json'
        }
    if content == 'record':
        return {
            'token': token,
            'content': content,
            'action': 'export',
            'format': 'csv',
            'type': 'flat',
            'csvDelimiter': '',
            'rawOrLabel': 'raw',
            'rawOrLabelHeaders': 'raw',
            'exportCheckboxLabel': 'false',
            'exportSurveyFields': 'false',
            'exportDataAccessGroups': 'false',
            'returnFormat': 'json'
        }
    return {
        'token': token,
        'content': content,
        'format': 'csv',
        'returnFormat': 'json'
    }

def request_redcap_data(
        token,
        content,
        redcap_link = 'https://mrprcbcw.hosts.jhmi.edu/redcap/api/',
    ):
    """
    send the data request to redcap
    
    Args:
        token: redcap token
        content: redcap table/form to use
        redcap_link: link to redcap
    """
    return requests.post(
        redcap_link,
        data=create_request_input(token, content),
    )

def get_redcap_df(token, content):
    """
    convert the text output from the redcap 
    request to pandas df

    Args:
        token: redcap token
        content: redcap table/form to use
    """
    return pandas.read_csv(
        io.StringIO(request_redcap_data(token, content).text), dtype= str,
    )

class DBUtils:
    """
    create a direct connection to the database. specifically
    helpful when we need to write a table to the database.
    More advanced permissions are needed for this.
    """

    def __init__(self):
        self.dbengine = None
        self.dbmeta = None
        self.creds = dict.fromkeys({"username", "password", "IP", "port"})

    def open_connection(self, database="AstroPathXfer"):
        """
        create a direct database connection and return
        the engine as well as the meta data object
        for that database.

        Args:
            creds: a dictionary with username, password,
                IP and port for the connection.
            database: database to connect to
        """
        #
        self.dbengine = create_engine(
            f"""mssql+pymssql://{self.creds['username']}:{
                self.creds['password']}@{self.creds['IP']}:{
                self.creds['port']}/{database}""",
        )
        self.dbmeta = MetaData()
        self.dbmeta.reflect(bind=self.dbengine)

    def write_to_database(
        self, data, table_name, if_exists="append", index=False
    ):
        """
        write data to the database

        Args:
            data: the data to write
            table_name: the table name to write to
            if_exists: how to handle the write if the table
                already exists
            index: whether or not to create an index on the table
        """
        if self.dbengine is None:
            self.open_connection()
        #
        if isinstance(data, geopandas.GeoDataFrame):
            data = geopandas.GeoDataFrame.to_wkt(data)
        #
        data.to_sql(
            name=table_name,
            con=self.dbengine,
            if_exists=if_exists,
            index=index,
            dtype = sqlalchemy.NVARCHAR(),
        )

def get_sql_connection(
        username = '',
        password = '',
        ip = '',
        port = '',
    ):
    """
    get a database connection. 
    # sql to check ip and port
    # select distinct local_net_address, local_tcp_port 
    # from sys.dm_exec_connections where local_net_address is not null

    Args:
        username: username for database access
        password: password for database access
        ip: ip address for database
        port: port for database
    """
    db = DBUtils()
    db.creds['IP'] = ip
    db.creds['port'] = port   
    db.creds['username'] = username
    db.creds['password'] = password
    db.open_connection()
    
    return db

def write_redcap_table_to_db(data, table_name, db = None, if_exists = 'replace'):
    """
    write table from redcap into the database, always append 'redcap_' to table name
    set to always replace the data in the table

    Args:
        data: the data to write into the table
        table_name: the name of the table to write 
            (note that 'redcap_' always appended to name)
        db: a DBUtils() object where [obj].open_connection() 
            has been run to init the connection
        if_exists: pandas.to_sql() if_exists behavior
    """
    if db is None:
        db = get_sql_connection()
    db.write_to_database(data, f"redcap_{table_name}", if_exists=if_exists)

def get_tier_records(
        tier, 
        metadata_data, 
        record_data, 
        key_1 = 'patient_tier',
        pt_id = 'astropt'
    ):
    """
    get the records for a specified tier, always append
    ``pt_id`` and drop the descrptive
    keys from the table to simplfy. Note that the
    ``key_1`` is NULL in the records table so the
    filter is a little different.

    Args:
        tier: the tier to get the records for
        metadata_data: the metadata redcap export
        record_data: the record redcap export
        key_1: the first key in the records table 
            ('patient_tier' default)
        pt_id: The patient identifier ('astropt' default)
    """
    columns = metadata_data.loc[
        (metadata_data['form_name'] == tier) & 
        (metadata_data['field_type'] != 'checkbox') & 
        (metadata_data['field_type'] != 'descriptive'),
        'field_name']
    #
    # handle checkboxes and ensure the column ordering is correct
    #
    columns_chckbox = metadata_data.loc[
        (metadata_data['form_name'] == tier) & 
        (metadata_data['field_type'] == 'checkbox'),
        'field_name']
    columns_chckbox += '___'
    columns_chckbox = [col for col in record_data if col.startswith(tuple(columns_chckbox))]
    columns_chckbox = pandas.Series(columns_chckbox)
    columns = pandas.concat([columns, columns_chckbox])
    #
    # add 'redcap_repeat_instance', 'redcap_repeat_instrument', and ``pt_id``
    # if they are not present
    #
    if pt_id not in columns.tolist():
        columns = pandas.Series(numpy.concatenate(([pt_id], columns.values)))
    #
    if 'redcap_repeat_instrument' not in columns.tolist():
        columns = pandas.Series(numpy.concatenate((['redcap_repeat_instrument'], columns.values)))
    #
    if 'redcap_repeat_instance' not in columns.tolist():
        columns = pandas.Series(numpy.concatenate((['redcap_repeat_instance'], columns.values)))
    #
    # add the '{tier}_complete' column
    #
    columns = pandas.Series(numpy.concatenate(([f"{tier}_complete"], columns.values)))
    #
    # The patient tier is not a repeat instrument and the column will be NULL, but
    # we want the tables to be consistent so we fill it in.
    #
    if tier == key_1:
        return record_data.loc[record_data['redcap_repeat_instrument'].isna(), record_data.columns.isin(columns)]
    #
    return record_data.loc[record_data['redcap_repeat_instrument'] == tier, record_data.columns.isin(columns)]

Finally, run the last cell. This will read the REDCap data from the 'instrument', 'metadata', 'record', and 'log' tables using the input ``token``. Next, it will build a database connection with the inputs provided above for ``username``, ``password``, ``ip``, and ``port``. Then it will write the 4 tables above into the SQL database in a context named `AstroPathXfer` prepending 'redcap_' to the names. Last, the code will filter the 'metadata' into separate tiers based on the REDCap 'instrument' (or AstroID tiers). The following edits are made to the tables before loading:
- Checkboxes are collapsed into a single field
- The 'desciptive' fields are filtered out
- The 'redcap_repeat_instance', 'redcap_repeat_instrument', and ``pt_id`` ('atroid') columns are prepended if they do not exist
- The patient tier (or ``key_1``) is not a repeat instrument and the column will be NULL, but we want the tables to be consistent in the database so we fill it in.

Once filtered, we load the tables in the SQL database naming them as 'redcap_``tier_name``'

In [11]:
# get redcap tables
instrument_data = get_redcap_df(token, 'instrument')
metadata_data = get_redcap_df(token, 'metadata')
record_data = get_redcap_df(token, 'record')
log_data = get_redcap_df(token, 'log')
# build database connection to database
db = get_sql_connection(username, password, ip, port)
# write basic tables
write_redcap_table_to_db(instrument_data, 'instrument', db = db)
write_redcap_table_to_db(metadata_data, 'metadata', db = db)
write_redcap_table_to_db(record_data, 'record', db = db)
write_redcap_table_to_db(log_data, 'log', db = db)
# filter records and write tables
for tier in instrument_data['instrument_name']:
    print(tier)
    data = get_tier_records(tier, metadata_data, record_data, key_1, pt_id)
    write_redcap_table_to_db(data, tier, db = db)

patient_tier
diagnosis_tier
clinical_tier
specimen_tier
block_tier
slide_tier
